## Imports and setup

In [12]:
# Grab data from ACS with cenpy (don't need to run every time!)
# Variables:
#    - total population (B01003_001E)
#    - total white (B02008_001E)
#    - total black (B02009_001E)
#    - total asian (B02011_001E)
#    - total hispanic (B03001_003E)
#    - household income (B19001_001E)
#    - median house price (B25077_001E)
#    - total tenure (B25032_001E)
#    - total owner occupied (B25032_002E)
#    - total renter occupied (B25032_007E)
#    - median contract rent (B25058_001E)
#    - total in labor force (B23025_002E)
#    - total employed (civilian) (B23025_004E)
#    - total unemployed (civilian) (B23025_005E)

countydata = gpd.GeoDataFrame()

states = pd.read_csv('../utils/states.csv')['State'].values
for state in tqdm(states):
    countydata = countydata.append(acs.from_state(state=state, variables=variables, level='county'))

countydata.to_csv('../data/ACS_countydata.csv')

100%|██████████| 51/51 [19:02<00:00, 22.41s/it]


In [1]:
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from scipy.linalg import norm

from tqdm import tqdm
from cenpy.products import ACS
from spint.gravity import Gravity, Attraction, Production

acs = ACS()
LA_FIPS = '06037'  # LA County FIPS code
variables = ['B01003_001E', 'B02008_001E', 'B02009_001E', 'B02011_001E', 'B03001_003E', \
    'B19001_001E', 'B25077_001E', 'B25032_001E', 'B25032_002E', 'B25032_007E', \
    'B25058_001E', 'B23025_002E', 'B23025_004E', 'B23025_005E']

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

## Load data

In [2]:
# Flows
inflows_raw = pd.read_csv('../data/LACounty_ACS_2014_2018_All_IN.csv', usecols=['State/County FIPS', 'Total', 'Margin of Error (+/-)']).replace("'", "", regex=True)
outflows_raw = pd.read_csv('../data/LACounty_ACS_2014_2018_All_OUT.csv', usecols=['State/County FIPS', 'Total', 'Margin of Error (+/-)']).replace("'", "", regex=True)
outflows_raw.rename(columns={'State/County FIPS' : 'destFIPS', 'Total' : 'total_out', 'Margin of Error (+/-)' : 'outMOE'}, inplace=True)
inflows_raw.rename(columns={'State/County FIPS' : 'originFIPS', 'Total' : 'total_in', 'Margin of Error (+/-)' : 'inMOE'}, inplace=True)

In [3]:
# County-level demographic data (and convert covariate columns)
countydata = gpd.read_file('../data/ACS_countydata.csv', GEOM_POSSIBLE_NAMES="geometry", KEEP_GEOM_COLUMNS="NO").replace('', 0)
for var in variables:
    countydata[var] = countydata[var].astype(float)

In [4]:
# Ping ACS for data for LA county
la_data = acs.from_county("Los Angeles County, CA", variables=variables, level='county')

In [5]:
# Convert races, employment to percents
countydata['pctwhite'] = countydata['B02008_001E'] / countydata['B01003_001E']
countydata['pctblack'] = countydata['B02009_001E'] / countydata['B01003_001E']
countydata['pctasian'] = countydata['B02011_001E'] / countydata['B01003_001E']
countydata['pcthispa'] = countydata['B03001_003E'] / countydata['B01003_001E']
countydata['pctemplo'] = countydata['B23025_004E'] / countydata['B01003_001E']
countydata['pctunemp'] = countydata['B23025_005E'] / countydata['B01003_001E']

la_data['pctwhite'] = la_data['B02008_001E'] / la_data['B01003_001E']
la_data['pctblack'] = la_data['B02009_001E'] / la_data['B01003_001E']
la_data['pctasian'] = la_data['B02011_001E'] / la_data['B01003_001E']
la_data['pcthispa'] = la_data['B03001_003E'] / la_data['B01003_001E']
la_data['pctemplo'] = la_data['B23025_004E'] / la_data['B01003_001E']
la_data['pctunemp'] = la_data['B23025_005E'] / la_data['B01003_001E']

In [6]:
plainvars = ['pctwhite', 'pctblack', 'pctasian', 'pcthispa', \
    'B19001_001E', 'B25077_001E', 'B25032_001E', 'B25032_002E', 'B25032_007E', \
    'B25058_001E', 'pctemplo', 'pctunemp']

## Rearrage data to prep for analysis

In [7]:
# Merge all the data into one big dataframe to make it simpler
data = pd.merge(pd.merge(countydata, outflows_raw, how='inner', left_on='GEOID', right_on='destFIPS'), inflows_raw, how='inner', left_on='GEOID', right_on='originFIPS').set_crs(epsg=3395)
la_covars = np.tile(la_data[plainvars].values, (data.shape[0], 1))  # repeat the data so that we account for the single location properly

In [8]:
# costs are all distances between LA and the out or in destination
coords = np.hstack((data.centroid.x.values.reshape(-1, 1), data.centroid.y.values.reshape(-1, 1)))
la_coords = np.array([la_data.centroid.x[0], la_data.centroid.y[0]])
cost = norm(la_coords - coords, axis=1)

## Calibrate model 

In [10]:
data[plainvars]

,pctwhite,pctblack,pctasian,pcthispa,B19001_001E,B25077_001E,B25032_001E,B25032_002E,B25032_007E,B25058_001E,pctemplo,pctunemp
0,0.571839,0.417700,0.002181,1.0,7620.0,91800.0,7620.0,5615.0,0.0,285.0,0.342241,0.047496
1,0.657423,0.317727,0.017904,1.0,71507.0,161700.0,71507.0,44920.0,27.0,624.0,0.444865,0.032139
2,0.823712,0.125613,0.024823,1.0,76868.0,199500.0,76868.0,61189.0,81.0,801.0,0.498718,0.022279
3,0.711386,0.277923,0.012276,1.0,39560.0,127100.0,39560.0,25570.0,10.0,531.0,0.423675,0.034983
4,0.878319,0.098628,0.013016,1.0,76133.0,182000.0,76133.0,55470.0,214.0,731.0,0.440239,0.025644
...,...,...,...,...,...,...,...,...,...,...,...,...
1297,0.764410,0.008302,0.003568,1.0,15167.0,189700.0,15167.0,10699.0,0.0,627.0,0.462755,0.029291
1298,0.980789,0.007504,0.009207,1.0,7063.0,205900.0,7063.0,5539.0,4.0,684.0,0.485552,0.030759
1299,0.952860,0.014261,0.013722,1.0,16269.0,201000.0,16269.0,12001.0,17.0,772.0,0.510679,0.027961
1300,0.929475,0.024668,0.043380,1.0,16009.0,223000.0,16009.0,7940.0,18.0,678.0,0.556478,0.027857


In [9]:
model = Gravity(flows=data['total_out'].values, o_vars=la_covars, d_vars=data[plainvars].values, cost=cost, cost_func='exp', constant=False)

ValueError: Zero values detected in column 1 of destination variables, which are undefined for Poisson log-linear spatial interaction models

In [ ]:
paramnames = [*[x + '_o' for x in variables], *[x + '_d' for x in variables], 'distance']
pd.DataFrame(data={'paramname' : paramnames, 'paramval' : model.params, 'SE' : model.std_err, 'tvalue' : model.tvalues, 'pvalue' : model.pvalues})  # intercept, origin attrs, dest attrs, distance

In [ ]:
model.SRMSE

In [ ]:
out_model = Production(flows=data['total_out'].values, origins=data['originFIPS'].values, d_vars=data[variables].values, cost=cost, cost_func='exp')
in_model = Attraction(flows=data['total_in'].values, destinations=data['destFIPS'].values, o_vars=data[variables].values, cost=cost, cost_func='exp')